# Demo training on cora, node classification

## Import modules

In [19]:
import gli
import torch
from torch import nn
import torch.nn.functional as F
from dgl.nn.pytorch import GraphConv
from gli.utils import to_dense

## load data

In [20]:
data = gli.dataloading.get_gli_dataset("cora", "NodeClassification")
g = data[0]
# convert sparse tensor to dense
g = to_dense(g)

## load features, labels, masks

In [21]:
features = g.ndata["NodeFeature"]
labels = g.ndata["NodeLabel"]
train_mask = g.ndata["train_mask"]
val_mask = g.ndata["val_mask"]
test_mask = g.ndata["test_mask"]

## print data statistics

In [22]:
in_feats = features.shape[1]
n_classes = data.num_labels
n_edges = g.number_of_edges()

print(f"""----Data statistics------'
    #Edges {n_edges}
    #Classes {n_classes}
    #Train samples {train_mask.int().sum().item()}
    #Val samples {val_mask.int().sum().item()}
    #Test samples {test_mask.int().sum().item()}""")

----Data statistics------'
    #Edges 10556
    #Classes 7
    #Train samples 140
    #Val samples 500
    #Test samples 1000


## Prepare evaluation function

In [23]:
def accuracy(logits, labels):
    """Calculate accuracy."""
    _, indices = torch.max(logits, dim=1)
    correct = torch.sum(indices == labels)
    return correct.item() * 1.0 / len(labels)


def evaluate(model, features, labels, mask, eval_func):
    """Evaluate model."""
    model.eval()
    with torch.no_grad():
        logits = model(features)
        logits = logits[mask]
        labels = labels[mask]
        return eval_func(logits, labels)

## Load a GCN model

In [24]:
class GCN(nn.Module):
    """GCN network."""

    def __init__(self,
                 g,
                 in_feats,
                 n_hidden,
                 n_classes,
                 n_layers,
                 activation,
                 dropout):
        """Initiate model."""
        super().__init__()
        self.g = g
        self.layers = nn.ModuleList()
        # input layer
        self.layers.append(GraphConv(in_feats, n_hidden,
                                     activation=activation))
        # hidden layers
        for _ in range(n_layers - 2):
            self.layers.append(GraphConv(n_hidden, n_hidden,
                                         activation=activation))
        # output layer
        self.layers.append(GraphConv(n_hidden, n_classes))
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, features):
        """Forward."""
        h = features
        for i, layer in enumerate(self.layers):
            if i != 0:
                h = self.dropout(h)
            h = layer(self.g, h)
        return h

model = GCN(g=g,
            in_feats=in_feats,
            n_hidden=8,
            n_classes=n_classes,
            n_layers=2,
            activation=F.relu,
            dropout=.6)

## Prepare optimizer, evaluation function and loss function

In [25]:
optimizer = torch.optim.AdamW(model.parameters(), lr=.01, weight_decay=.001)
eval_func = accuracy
loss_fcn = nn.CrossEntropyLoss()

## Train the model for 200 epoches

In [26]:
for epoch in range(200):
        model.train()

        # forward
        logits = model(features)
        loss = loss_fcn(logits[train_mask], labels[train_mask])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_acc = eval_func(logits[train_mask], labels[train_mask])
        val_acc = evaluate(model, features, labels, val_mask, eval_func)
        print(f"Epoch {epoch:05d} | Loss {loss.item():.4f} | "
              f"TrainAcc {train_acc:.4f} | ValAcc {val_acc:.4f}")

test_acc = evaluate(model, features, labels, test_mask, eval_func)
print(f"Test Accuracy: {test_acc:.4f}")


Epoch 00000 | Loss 1.9457 | TrainAcc 0.1000 | ValAcc 0.0760
Epoch 00001 | Loss 1.9419 | TrainAcc 0.2071 | ValAcc 0.1040
Epoch 00002 | Loss 1.9352 | TrainAcc 0.3286 | ValAcc 0.1940
Epoch 00003 | Loss 1.9299 | TrainAcc 0.3571 | ValAcc 0.2820
Epoch 00004 | Loss 1.9232 | TrainAcc 0.4071 | ValAcc 0.3440
Epoch 00005 | Loss 1.9170 | TrainAcc 0.4214 | ValAcc 0.4180
Epoch 00006 | Loss 1.9054 | TrainAcc 0.4571 | ValAcc 0.4580
Epoch 00007 | Loss 1.9073 | TrainAcc 0.3571 | ValAcc 0.4940
Epoch 00008 | Loss 1.8970 | TrainAcc 0.4214 | ValAcc 0.5420
Epoch 00009 | Loss 1.8878 | TrainAcc 0.4643 | ValAcc 0.5260
Epoch 00010 | Loss 1.8813 | TrainAcc 0.4714 | ValAcc 0.4900
Epoch 00011 | Loss 1.8766 | TrainAcc 0.4357 | ValAcc 0.4940
Epoch 00012 | Loss 1.8466 | TrainAcc 0.5143 | ValAcc 0.4900
Epoch 00013 | Loss 1.8469 | TrainAcc 0.5071 | ValAcc 0.4960
Epoch 00014 | Loss 1.8260 | TrainAcc 0.4857 | ValAcc 0.5120
Epoch 00015 | Loss 1.8118 | TrainAcc 0.5857 | ValAcc 0.5400
Epoch 00016 | Loss 1.8273 | TrainAcc 0.4